PROCESO DE MERGE ENAHO 2024 - PERÚ
==============================================
Autor: Mateo Becerra Restrepo
Propósito: Unión de módulos 1, 2, 3, 4, 5 y 34

ESTRUCTURA DE LA UNIÓN:
1. Vivienda + Sumarias -> Nivel hogar (1:1)
2. Hogar + Personas -> Relación 1:m
3. Base + Educación, Salud, Empleo -> Nivel personas (1:1)

VARIABLES CLAVE:
- Identificación: conglome, vivienda, hogar, codperso
- Ponderación: factor07
- Estratificación: dominio, estrato
- Socioeconómica: pobreza

NOTAS TÉCNICAS:
- Merge left para módulos opcionales (educación, salud, empleo)
- Validación de cardinalidad en cada paso
- Base final: [N] observaciones a nivel persona-hogar

ARCHIVOS ORIGINALES:
- enaho01-2024-100.dta (Vivienda)
- enaho01-2024-200.dta (Personas)
- sumaria-2024.dta (Sumaria)
- enaho01a-2024-300.dta (Educación)
- enaho01a-2024-400.dta (Salud) 
- enaho01a-2024-500.dta (Empleo)


In [ ]:
#Cargamos librerías
import pandas as pd
import os
import pyreadstat
import numpy as np
import weightedcalcs as wc

ruta_base = r"D:\Mateo\ICSI\ENAHO\2024\DTA"
os.chdir(ruta_base)
modulos = {
    'vivienda': 'enaho01-2024-100.dta',
    'personas': 'enaho01-2024-200.dta',
    'educacion': 'enaho01a-2024-300.dta',
    'salud': 'enaho01a-2024-400.dta',
    'empleo_ingresos': 'enaho01a-2024-500.dta',
    'sumarias': 'sumaria-2024.dta',
}

#Crear función para cargar módulos
def cargar_modulos(archivo):
    df, meta  = pyreadstat.read_dta(os.path.join(ruta_base, archivo))
    return df

# Cargar todos los módulos
df_vivienda = cargar_modulos(modulos['vivienda'])
df_personas = cargar_modulos(modulos['personas'])
df_educacion = cargar_modulos(modulos['educacion'])
df_empleo = cargar_modulos(modulos['empleo_ingresos'])
df_sumarias = cargar_modulos(modulos['sumarias'])

df_hogar = pd.merge(
    df_vivienda,
    df_sumarias,
    on=['conglome', 'vivienda', 'hogar'],
    how='inner',
    suffixes=('_sum', '_viv')
)

df_base = pd.merge(
    df_hogar,
    df_personas,
    on=['conglome', 'vivienda', 'hogar'],
    how='inner',
    suffixes=('_hogar', '_per')
)

df_base = pd.merge(
    df_base,
    df_educacion,
    on=['conglome', 'vivienda', 'hogar', 'codperso'],
    how='left'
)

df_base = pd.merge(
    df_base,
    df_empleo,
    on=['conglome', 'vivienda', 'hogar', 'codperso'],
    how='left'
)

#Crear dptos
df_base['cod_dpto'] = df_base['ubigeo_sum'].str[:2]
nombre_dpto = {
    '01': 'Amazonas',
    '02': 'Áncash',
    '03': 'Apurímac',
    '04': 'Arequipa',
    '05': 'Ayacucho',
    '06': 'Cajamarca',
    '07': 'Callao',
    '08': 'Cusco',
    '09': 'Huancavelica',
    '10': 'Huánuco',
    '11': 'Ica',
    '12': 'Junín',
    '13': 'La Libertad',
    '14': 'Lambayeque',
    '15': 'Lima',
    '16': 'Loreto',
    '17': 'Madre de Dios',
    '18': 'Moquegua',
    '19': 'Pasco',
    '20': 'Piura',
    '21': 'Puno',
    '22': 'San Martín',
    '23': 'Tacna',
    '24': 'Tumbes',
    '25': 'Ucayali'
}
df_base['dpto'] = df_base['cod_dpto'].map(nombre_dpto)
variables_interes = ['aÑo_viv', 'mes_viv', 'ubigeo_viv', 'dominio_viv', 'estrato_viv' , 'conglome',
                     'vivienda', 'hogar', 'p101', 'p102', 'p103', 'p103a', 'p105a', 'p110', 'p110a',
                     'p110c', 'p110f', 'p111a', 'p1121', 'p1123', 'p1124', 'p1125', 'p1126', 'p1127',
                     'p113a', 'p203_x', 'p204_x', 'p205_x', 'p206_x', 'p207_x', 'p208a_x', 'p209_x', 'p210', 'p300a',
                     'p301a_x', 'p302', 'p307', 'p308a', 'p500i', 'p501', 'p507', 'p511a', 'p524a1',
                     'p558a1', 'p558a2', 'p558a3', 'p558a4', 'p558a5', 'p558c', 'factor07', 'facpob07',
                     'factora07', 'fac500a','cod_dpto', 'dpto' , 'codinfor_x']

df_filtrado = df_base[variables_interes].copy()

#Crear variable Area

df_filtrado['área'] = np.where(
    df_filtrado['estrato_viv'] <= 5, 1,
    np.where(
        (df_filtrado['estrato_viv'] >= 6) & (df_filtrado['estrato_viv'] <= 8), 2, np.nan
    )
)

df_filtrado['área'] = df_filtrado['área'].map({1 : 'Urbana', 2: 'Rural'}) 
#Crear variable Dominio
df_filtrado['regnat'] = df_filtrado['dominio_viv']
df_filtrado['regnat'] = np.select(
    [
        df_filtrado['dominio_viv'].isin([1,2,3,8]),
        df_filtrado['dominio_viv'].isin([4,5,6]),
        df_filtrado['dominio_viv'] == 7
    ],
    [1,2,3],
    default=np.nan
)

df_filtrado['regnat'] = df_filtrado['regnat'].map({1 : 'Costa', 2: 'Sierra', 3: 'Selva', 0: 'Otros'}) 

Tasa neta de asistencia escolar 

In [191]:
#Pasar a numéricas las variables de interés
df_filtrado['mes_viv'] = pd.to_numeric(df_filtrado['mes_viv'], errors='coerce')
df_filtrado['p308a'] = pd.to_numeric(df_filtrado['p308a'], errors='coerce')
df_filtrado['p307'] = pd.to_numeric(df_filtrado['p307'], errors='coerce')
df_filtrado['p203_x'] = pd.to_numeric(df_filtrado['p203_x'], errors='coerce')
df_filtrado['p204_x'] = pd.to_numeric(df_filtrado['p204_x'], errors='coerce')
df_filtrado['p208a_x'] = pd.to_numeric(df_filtrado['p208a_x'], errors='coerce')

condicion_1 = (
    (df_filtrado['codinfor_x'] != "00") &
    (df_filtrado['p204_x'] == 1) &
    (~df_filtrado['p203_x'].isin([8,9])) &
    (df_filtrado['mes_viv'] >= 4) &
    (df_filtrado['p208a_x'].between(3,5))
)

df_3a5 = df_filtrado[condicion_1].copy()

df_3a5['tna_ini'] = 0

condicion_2 = (
    (df_3a5['p307'] == 1) &
    (df_3a5['p308a'] == 1)
)

df_3a5.loc[condicion_2, 'tna_ini'] = 1

calc = wc.Calculator("factora07")
media_total = calc.mean(df_3a5, "tna_ini")

media_area = df_3a5.groupby('área').apply(
    lambda x: calc.mean(x, "tna_ini")
)

media_regnat = df_3a5.groupby('regnat').apply(
    lambda x: calc.mean(x, "tna_ini")
)

C:\Users\mateo\AppData\Local\Temp\ipykernel_18344\2533864070.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_area = df_3a5.groupby('área').apply(
C:\Users\mateo\AppData\Local\Temp\ipykernel_18344\2533864070.py:35: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_regnat = df_3a5.groupby('regnat').apply(


Output dpto y área

In [199]:
ct_area = pd.crosstab(
    index=df_3a5['dpto'],
    columns=df_3a5['área'],
    values=df_3a5['factora07'],
    aggfunc='sum',
    normalize='index'
)

fila_total = pd.DataFrame({
    col: media_total for col in ct_area.columns
}, index=['Total'])

# Concatenar al final de la crosstab
ct_area = pd.concat([ct_area, fila_total])
print(ct_area.round(3)*100)

               Rural  Urbana
Amazonas        45.3    54.7
Apurímac        56.3    43.7
Arequipa         5.9    94.1
Ayacucho        38.7    61.3
Cajamarca       54.1    45.9
Callao           0.0   100.0
Cusco           44.9    55.1
Huancavelica    76.8    23.2
Huánuco         66.0    34.0
Ica              4.6    95.4
Junín           33.9    66.1
La Libertad     22.1    77.9
Lambayeque      14.8    85.2
Lima             1.1    98.9
Loreto          40.4    59.6
Madre de Dios   12.0    88.0
Moquegua        11.0    89.0
Pasco           36.6    63.4
Piura           20.9    79.1
Puno            36.8    63.2
San Martín      38.7    61.3
Tacna           17.0    83.0
Tumbes           1.3    98.7
Ucayali         19.3    80.7
Áncash          33.2    66.8
Total           81.8    81.8
